# Import Libraries

In [5]:
from prefect import Flow, task
from prefect.tasks.shell import ShellTask
from prefect.schedules.clocks import IntervalClock
from prefect.schedules import Schedule
from dask.distributed import Client
from prefect.executors import DaskExecutor
from datetime import datetime, timedelta
import pendulum
import os
from prefect.run_configs import LocalRun


cwd = os.getcwd().split("/")[-1]
if cwd == "notebooks":
    os.chdir("../")

In [6]:
client = Client("192.168.1.26:8786")
dask_address = str(client.scheduler).split("'")[1]
executor = DaskExecutor(address=dask_address)
client

Client Scheduler: tcp://192.168.1.26:8786 Dashboard: http://192.168.1.26:8787/status,Cluster Workers: 6 Cores: 6 Memory: 34.36 GB


# Scrape Events Flow

In [12]:
event_schedule = Schedule([IntervalClock(start_date=pendulum.datetime(2020, 12, 13, 6, tz="America/Los_Angeles"),
                               interval=timedelta(days=7))])

shell_task = ShellTask(helper_script="cd scrapy_ufcstats")

with Flow("Scrape Events", executor=executor, schedule=event_schedule) as f: 
    date_today = datetime.now().date()
    scrapy_crawl_events_cmd = f"scrapy crawl events -O ../data/events/event_urls_{date_today}.csv"
    shell_task(command=scrapy_crawl_events_cmd)
f.register("scrape-ufcstats")


Result check: OK
Flow URL: http://localhost:8080/default/flow/591dbcfe-2051-44d2-9901-68f435aefc82
 └── ID: db52be27-164e-4d57-b530-f10888264251
 └── Project: scrape-ufcstats
 └── Labels: ['Mac-mini.local']


'db52be27-164e-4d57-b530-f10888264251'

In [8]:
my_task = ShellTask(helper_script="cd scrapy_ufcstats", return_all=True, stream_output=True)

with Flow("test", executor=executor) as f:
    output = my_task.run(command="ls")
    print(output)

flow_state = f.run() # flow_state is a `State` object


['environment.yml', 'scrapy.cfg', 'scrapy_ufcstats']
[2021-01-05 20:26:03-0800] INFO - prefect.FlowRunner | Beginning Flow run for 'test'


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://192.168.1.26:60439 remote=tcp://192.168.1.26:8786>


[2021-01-05 20:26:03-0800] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
